# Attach polygons to AnnData

This notebook loads per-section h5ad files and attaches the exported polygons.json
to `adata.uns['polygons']`.


In [2]:
from pathlib import Path
import json
import scanpy as sc


## Configure paths


In [3]:
data_root = Path('/Volumes/processing2/nature-dev-mouse-reanalysis/data')
ad_all_path = Path('/Volumes/processing2/nature-dev-mouse-reanalysis/ad_all_processed.h5ad')

out_path = ad_all_path.with_name(ad_all_path.stem + '_with_polygons.h5ad')


## Attach polygons


In [ ]:
# Identify highly variable genes (optional)
import scanpy as sc

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000, flavor='seurat_v3')
adata


In [4]:
import numpy as np
from matplotlib.path import Path as MplPath

def polygons_path_for_section(section_name: str) -> Path:
    return data_root / section_name / 'polygons.json'

def load_polygons(section_name: str):
    poly_path = polygons_path_for_section(section_name)
    if not poly_path.exists():
        return None
    with open(poly_path, 'r') as f:
        return json.load(f)

def normalize_polygons(poly_val):
    # Returns list of polygons, each as (N,2) array
    if not isinstance(poly_val, list) or len(poly_val) == 0:
        return []
    # If this is a list of points -> single polygon
    if isinstance(poly_val[0], (list, tuple)) and len(poly_val[0]) == 2 and not isinstance(poly_val[0][0], (list, tuple)):
        return [np.array(poly_val, dtype=float)]
    # If this is a list of polygons (list of list of points)
    if isinstance(poly_val[0], (list, tuple)) and len(poly_val[0]) > 0 and isinstance(poly_val[0][0], (list, tuple)):
        return [np.array(p, dtype=float) for p in poly_val]
    return []

def get_cell_xy(adata):
    if 'x' in adata.obs.columns and 'y' in adata.obs.columns:
        return adata.obs[['x', 'y']].to_numpy(dtype=float)
    if 'spatial' in adata.obsm:
        return np.asarray(adata.obsm['spatial'], dtype=float)[:, :2]
    raise ValueError("No x/y in adata.obs and no adata.obsm['spatial'] found")

adata = sc.read_h5ad(ad_all_path)

# Build a dict of polygons by sample_id and attach to .uns
if 'sample_id' not in adata.obs.columns:
    raise ValueError('sample_id not found in adata.obs; re-run 02-baysor-to-h5ad.ipynb')

sample_ids = sorted(adata.obs['sample_id'].astype(str).unique())
polygons_by_section = {}

# Prepare output column for single-label regions
adata.obs['anatomical_region'] = ''

# Precompute coordinates for all cells
all_xy = get_cell_xy(adata)

for sample_id in sample_ids:
    poly = load_polygons(sample_id)
    if poly is None:
        print(f'Skipping missing polygons for {sample_id}')
        continue
    polygons_by_section[sample_id] = poly

    # Mask for cells in this sample
    mask = adata.obs['sample_id'].astype(str) == sample_id
    idx = np.where(mask)[0]
    if idx.size == 0:
        continue
    xy = all_xy[idx]

    # First-match assignment (leave empty if no polygon hit)
    labels = ['' for _ in range(xy.shape[0])]
    assigned = np.zeros(xy.shape[0], dtype=bool)

    for region_name, poly_val in poly.items():
        for poly_arr in normalize_polygons(poly_val):
            if poly_arr.shape[0] < 3:
                continue
            path = MplPath(poly_arr)
            inside = path.contains_points(xy)
            for i, hit in enumerate(inside):
                if hit and not assigned[i]:
                    labels[i] = region_name
                    assigned[i] = True

    adata.obs.loc[mask, 'anatomical_region'] = labels

adata.uns['polygons_by_section'] = polygons_by_section
adata.uns['polygons_root'] = str(data_root)

adata.write(out_path)
out_path


Skipping missing polygons for section_02F
Skipping missing polygons for section_06F


PosixPath('/Volumes/processing2/nature-dev-mouse-reanalysis/ad_all_processed_with_polygons.h5ad')

In [5]:
adata.obs.anatomical_region.unique()

['', 'm1', 'm1R', '(OV)', 'r4', ..., 'PO', 'PHy', 'm1F', 'm1B', 'Tel']
Length: 25
Categories (25, object): ['', '(OV)', 'H', 'PHy', ..., 'r1', 'r2', 'r3', 'r4']

In [6]:
adata.uns['log1p']

{'base': None}

In [11]:
del adata.uns['rank_genes_groups']['params'] 

In [16]:
sc.pp.highly_variable_genes(adata,layer='normalized', n_top_genes=50, flavor='seurat_v3')


/Users/christoffer/miniconda3/envs/cellcharter-env/lib/python3.12/site-packages/legacy_api_wrap/__init__.py:88: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  return fn(*args_all, **kw)


In [20]:
adata.X = adata.layers['normalized'].copy()

In [21]:
adata.write(out_path)